In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import math

# Initialize empty board
board = [[' ' for _ in range(3)] for _ in range(3)]
colors = {'X': 'red', 'O': 'blue', ' ': 'white', 'PRUNED': 'gray'}

def plot_board(b, evals=None, pruned=None):
    clear_output(wait=True)
    fig, ax = plt.subplots(figsize=(5,5))
    # Draw symbols
    for i in range(3):
        for j in range(3):
            val = b[i][j]
            color = colors[val] if val != ' ' else 'black'
            ax.text(j+0.5, 2-i+0.5, val, fontsize=40, ha='center', va='center', color=color)
    # Draw evaluations
    if evals:
        for (i,j), score in evals.items():
            ax.text(j+0.5, 2-i+0.2, f"{score}", fontsize=12, ha='center', va='center', color='green')
    # Draw pruned marks
    if pruned:
        for (i,j) in pruned:
            ax.text(j+0.5, 2-i+0.2, "×", fontsize=12, ha='center', va='center', color='gray')

    ax.set_xlim(0,3)
    ax.set_ylim(0,3)
    ax.set_xticks(range(4))
    ax.set_yticks(range(4))
    ax.grid(True)
    plt.show()
    time.sleep(0.7)

def is_moves_left(b):
    return any(cell == ' ' for row in b for cell in row)

def evaluate(b):
    for i in range(3):
        if b[i][0] == b[i][1] == b[i][2] != ' ':
            return 10 if b[i][0]=='X' else -10
        if b[0][i] == b[1][i] == b[2][i] != ' ':
            return 10 if b[0][i]=='X' else -10
    if b[0][0] == b[1][1] == b[2][2] != ' ':
        return 10 if b[0][0]=='X' else -10
    if b[0][2] == b[1][1] == b[2][0] != ' ':
        return 10 if b[0][2]=='X' else -10
    return 0

# Minimax with alpha-beta
def minimax(b, depth, is_max, alpha, beta, pruned_moves=None, evals=None):
    score = evaluate(b)
    if score != 0 or not is_moves_left(b):
        return score

    if is_max:
        max_eval = -math.inf
        for i in range(3):
            for j in range(3):
                if b[i][j]==' ':
                    b[i][j] = 'X'
                    val = minimax(b, depth+1, False, alpha, beta, pruned_moves, evals)
                    b[i][j] = ' '
                    max_eval = max(max_eval, val)
                    alpha = max(alpha, val)
                    if evals is not None:
                        evals[(i,j)] = val
                    if beta <= alpha:
                        if pruned_moves is not None:
                            pruned_moves.append((i,j))
                        break
        return max_eval
    else:
        min_eval = math.inf
        for i in range(3):
            for j in range(3):
                if b[i][j]==' ':
                    b[i][j] = 'O'
                    val = minimax(b, depth+1, True, alpha, beta, pruned_moves, evals)
                    b[i][j] = ' '
                    min_eval = min(min_eval, val)
                    beta = min(beta, val)
                    if evals is not None:
                        evals[(i,j)] = val
                    if beta <= alpha:
                        if pruned_moves is not None:
                            pruned_moves.append((i,j))
                        break
        return min_eval

def find_best_move(b):
    best_val = -math.inf
    best_move = (-1,-1)
    evals = {}
    pruned = []

    for i in range(3):
        for j in range(3):
            if b[i][j]==' ':
                b[i][j] = 'X'
                move_val = minimax(b, 0, False, -math.inf, math.inf, pruned, evals)
                b[i][j] = ' '
                evals[(i,j)] = move_val
                if move_val > best_val:
                    best_val = move_val
                    best_move = (i,j)
    plot_board(b, evals, pruned)  # show evals + pruned moves
    return best_move

# Animated game loop
def play_game():
    plot_board(board)
    while True:
        if not is_moves_left(board) or evaluate(board)!=0:
            break

        # Player move
        row = int(input("Enter row (0-2): "))
        col = int(input("Enter col (0-2): "))
        if board[row][col]!=' ':
            print("Invalid! Try again.")
            continue
        board[row][col]='O'
        plot_board(board)

        # AI move
        if is_moves_left(board) and evaluate(board)==0:
            ai_move = find_best_move(board)
            board[ai_move[0]][ai_move[1]]='X'
            print(f"AI moves at {ai_move}")
            plot_board(board)

    # Game over
    score = evaluate(board)
    if score==10:
        print("AI Wins!")
    elif score==-10:
        print("You Win!")
    else:
        print("Draw!")

# Uncomment below to play
play_game()